# Creating the proper techno dataset

This notebook is used to organize the audio files by class. We are going to download the proper techno spotify playlist and use that as the proper techno true labels. For the negative classes we will take some hard techno, trance, melodic techno spotify playlists as the negatives


In [1]:
!pwd

/Users/zein/proper-classifier


## EDA

In [ ]:
# need to cd into the directory first..
# testing librosa library on one audio file, setting the params for reading each audio file

file_name = ''
audio_data, sampling_rate = librosa.load(file_name, mono=False) 
librosa.display.waveplot(audio_data,sr=sampling_rate)
ipd.Audio(file_name)



## Load into 2 column pandas dataframe: file name (id), label

## Extract the features over each file and create pandas dataframe with: filename (id), features, label

In [ ]:
def features_extractor(file):
    audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
    
    return mfccs_scaled_features

extracted_features=[]
for index_num,row in tqdm(metadata.iterrows()):
    file_name = os.path.join(os.path.abspath(audio_dataset_path),'fold'+str(row["fold"])+'/',str(row["slice_file_name"]))
    final_class_labels=row["class"]
    data=features_extractor(file_name)
    extracted_features.append([data,final_class_labels])

extracted_features_df=pd.DataFrame(extracted_features,columns=['feature','class'])
extracted_features_df.head(10)

## Create training, validation, and test splits

In [ ]:
X = np.array(extracted_features_df['feature'].tolist())
y = np.array(extracted_features_df['class'].tolist())

In [ ]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

labelencoder = LabelEncoder()
y = to_categorical(labelencoder.fit_transform(y))

## Save data splits into dataframes

In [ ]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X, y, test_size=0.2, random_state=0)